# Analysis of PELT signal error due to skip last window update

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-02-17 19:51:33,920 INFO    : root         : Using LISA logging configuration:
2017-02-17 19:51:33,922 INFO    : root         :   /data/Code/lisa/logging.conf


In [60]:
# Generate plots inline
%matplotlib inline
import json
import os
from trace import Trace
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import trappy

path_to_dat = "/home/joelaf/repo/lisa-aosp/external/lisa/ipynb/scratchpad/pelt-error/trace.dat"

## Parse Trace and Profiling Data

In [3]:
trace = Trace(None, path_to_dat, events=[ 'sched_switch', 'pelt_update' ])

## Trace visualization

In [4]:
trappy.plotter.plot_trace(trace.ftrace)

## Latency DataFrames

In [64]:
df = trace.data_frame.trace_event('pelt_update')
rq_df = df[df.cfs_rq == 1]
rq_df.columns

Index([u'__comm', u'__cpu', u'__line', u'__pid', u'__tgid', u'acc_load_avg',
       u'acc_util_avg', u'cfs_rq', u'delta_us', u'load_avg', u'load_err',
       u'load_sum', u'sum_err', u'util_avg', u'util_err'],
      dtype='object')

In [61]:
# Plot the accurate and the actual signals for the RQ
print 'UTIL ERROR'
trappy.ILinePlot(trace.ftrace,
                signals = [
                    'pelt_update:util_avg',
                    'pelt_update:acc_util_avg',
                ]).view()

print 'LOAD ERROR'
trappy.ILinePlot(trace.ftrace,
                signals = [
                    'pelt_update:load_avg',
                    'pelt_update:acc_load_avg',
                ]).view()

UTIL ERROR


""
""


LOAD ERROR


""
""


Data showing util/load errors on the RQ (cpu 1) when thread0 running
-------------------------------
Note that, as expected, the error exists only for cases where delta is < 1ms (now - last_update_time)

In [83]:
err_df = rq_df[(rq_df.util_err > 1) | (rq_df.load_err > 1)][rq_df['__comm'] == 'thread0']
err_df = err_df[['__comm',  'acc_load_avg', 'util_avg', 'acc_util_avg', \
                 'util_err', 'delta_us', 'load_sum', 'sum_err', 'load_avg', 'load_err']]

err_df.sort(columns=['delta_us'])
print 'number of errors: ' + str(len(err_df))
err_df


number of errors: 51


/home/joelaf/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/home/joelaf/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,__comm,acc_load_avg,util_avg,acc_util_avg,util_err,delta_us,load_sum,sum_err,load_avg,load_err
Time,,,,,,,,,,
0.212311,thread0,108,83,87,4,194,5164783,-198656,103,5
0.212334,thread0,108,83,88,5,26,5191407,-26624,103,5
0.232412,thread0,411,395,398,3,106,19645325,-108544,409,2
0.615666,thread0,81,74,79,5,236,3880110,-241664,75,6
0.615689,thread0,81,74,80,6,27,3907758,-27648,75,6
0.716483,thread0,80,76,79,3,122,3846510,-124928,76,4
0.736533,thread0,401,380,398,18,854,19171804,-874496,383,18
1.242656,thread0,403,396,402,6,276,19244596,-282624,397,6
1.749170,thread0,403,399,402,3,121,19240708,-123904,400,3


# Summary of issues

### * At 5.98s, there is a 6% error in util_avg 225 vs 211) - this causes a glitch and makes the signal less smooth
----
![glitch](pelt-signal-glitch-error-6pc.png)


### * At 3.06s, there is a 3% error in util_avg - causing lowered peak of util_avg (397 -> 387) with delta ~450us
![lower-peak](pelt-3percent-error.png)
